In [12]:
!pip install GingerIt  #used for spelling correction

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.7/99.7 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 5.1 MB/s eta 0:00:00


import libraries

In [1]:
import nltk
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

load data

In [2]:
import pandas as pd
df = pd.read_csv("/content/test.csv", encoding='unicode_escape')

In [3]:
df.head()

,Questions,Answers
0,What is data analytics?,Data analytics is the process of examining raw...
1,What are the main methods used in data analytics?,Data analytics encompasses various methods suc...
2,How is data analytics used in business?,Data analytics is widely used in business to g...
3,What are the key steps in the data analytics p...,The data analytics process typically involves ...
4,What is the role of machine learning in data a...,Machine learning plays a crucial role in data ...


convert to list

In [4]:
questions_list = df['Questions'].tolist()

In [5]:
answers_list = df['Answers'].tolist()

feature engineering

In [6]:
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.corpus import stopwords
import re

def preprocess(text):
    lemmatizer = WordNetLemmatizer()
    stemmer = PorterStemmer()
    text = re.sub(r'[^\w\s]', '', text)  # Remove non-alphanumeric characters
    tokens = nltk.word_tokenize(text.lower())
    tokens = [token for token in tokens if token not in stopwords.words('english')]
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    stemmed_tokens = [stemmer.stem(token) for token in lemmatized_tokens]
    return ' '.join(stemmed_tokens)

In [7]:
def preprocess_with_stopwords(text):
    lemmatizer = WordNetLemmatizer()
    stemmer = PorterStemmer()
    text = re.sub(r'[^\w\s]', '', text)  # Remove non-alphanumeric characters
    tokens = nltk.word_tokenize(text.lower())
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    stemmed_tokens = [stemmer.stem(token) for token in lemmatized_tokens]
    return ' '.join(stemmed_tokens)

In [8]:
vectorizer = TfidfVectorizer(tokenizer=nltk.word_tokenize)
X = vectorizer.fit_transform([preprocess(q) for q in questions_list])

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [9]:
vectorizer = TfidfVectorizer(tokenizer=nltk.word_tokenize)
X = vectorizer.fit_transform([preprocess(q) for q in questions_list])

def get_response(text):
    processed_text = preprocess_with_stopwords(text)
    print("processed_text:", processed_text)
    vectorized_text = vectorizer.transform([processed_text])
    similarities = cosine_similarity(vectorized_text, X)
    print("similarities:", similarities)
    max_similarity = np.max(similarities)
    print("max_similarity:", max_similarity)
    if max_similarity > 0.6:
        high_similarity_questions = [q for q, s in zip(questions_list, similarities[0]) if s > 0.6]
        print("high_similarity_questions:", high_similarity_questions)

        target_answers = []
        for q in high_similarity_questions:
            q_index = questions_list.index(q)
            target_answers.append(answers_list[q_index])
        print(target_answers)

        Z = vectorizer.fit_transform([preprocess_with_stopwords(q) for q in high_similarity_questions])
        processed_text_with_stopwords = preprocess_with_stopwords(text)
        print("processed_text_with_stopwords:", processed_text_with_stopwords)
        vectorized_text_with_stopwords = vectorizer.transform([processed_text_with_stopwords])
        final_similarities = cosine_similarity(vectorized_text_with_stopwords, Z)
        closest = np.argmax(final_similarities)
        return target_answers[closest]
    else:
        return "I can't answer this question."

get_response('Who is ms dhoni?')

processed_text: who is m dhoni
similarities: [[0. 0. 0. 0. 0. 0. 0. 0.]]
max_similarity: 0.0


"I can't answer this question."

In [10]:
vectorizer = TfidfVectorizer(tokenizer=nltk.word_tokenize)
X = vectorizer.fit_transform([preprocess(q) for q in questions_list])

def get_response(text):
    processed_text = preprocess_with_stopwords(text)
    print("processed_text:", processed_text)
    vectorized_text = vectorizer.transform([processed_text])
    similarities = cosine_similarity(vectorized_text, X)
    print("similarities:", similarities)
    max_similarity = np.max(similarities)
    print("max_similarity:", max_similarity)
    if max_similarity > 0.6:
        high_similarity_questions = [q for q, s in zip(questions_list, similarities[0]) if s > 0.6]
        print("high_similarity_questions:", high_similarity_questions)

        target_answers = []
        for q in high_similarity_questions:
            q_index = questions_list.index(q)
            target_answers.append(answers_list[q_index])
        print(target_answers)

        Z = vectorizer.fit_transform([preprocess_with_stopwords(q) for q in high_similarity_questions])
        processed_text_with_stopwords = preprocess_with_stopwords(text)
        print("processed_text_with_stopwords:", processed_text_with_stopwords)
        vectorized_text_with_stopwords = vectorizer.transform([processed_text_with_stopwords])
        final_similarities = cosine_similarity(vectorized_text_with_stopwords, Z)
        closest = np.argmax(final_similarities)
        return target_answers[closest]
    else:
        return "I can't answer this question."

get_response('what is machine learning')

processed_text: what is machin learn
similarities: [[0.         0.         0.         0.         0.77627227 0.
  0.         0.        ]]
max_similarity: 0.7762722680124386
high_similarity_questions: ['What is the role of machine learning in data analytics?']
['Machine learning plays a crucial role in data analytics by enabling the development of algorithms that can automatically learn from data and make predictions or take actions without being explicitly programmed. It is used for tasks such as classification, regression, clustering, and anomaly detection.']
processed_text_with_stopwords: what is machin learn


'Machine learning plays a crucial role in data analytics by enabling the development of algorithms that can automatically learn from data and make predictions or take actions without being explicitly programmed. It is used for tasks such as classification, regression, clustering, and anomaly detection.'

In [13]:
from gingerit.gingerit import GingerIt

text = 'What is Data Anlytics'

parser = GingerIt()
corrected_text = parser.parse(text)

print(corrected_text['result'])

What is Data Anlytics?
